<a href="https://colab.research.google.com/github/Rudr16a/SearchFile/blob/main/SearchFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import pandas as pd
from google.colab import drive

def search_function_in_sql_files(function_name, source_file_name):
    """
    Search for a specific SQL function in all SQL files across the SearchFile folders

    Args:
        function_name: The name of the function to search for
        source_file_name: The name of the file where the function is expected to be defined

    Returns:
        Dict with results from each folder
    """

    drive.mount('/content/drive')


    base_path = '/content/drive/MyDrive/SearchFile'


    if not os.path.exists(base_path):
        print(f"SearchFile folder not found at: {base_path}")
        return None, False


    all_folders = [folder for folder in os.listdir(base_path)
                  if os.path.isdir(os.path.join(base_path, folder))]


    required_folders = ['Functions', 'Procedure', 'Tables']
    for folder in required_folders:
        if folder not in all_folders:
            print(f"Required folder '{folder}' not found in SearchFile")


    results = {folder: [] for folder in all_folders}


    def check_file_for_function(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

                pattern = rf'\b{re.escape(function_name)}\b'
                return bool(re.search(pattern, content))
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return False


    source_file_path = os.path.join(base_path, 'Functions', source_file_name)
    function_exists_in_source = False

    if os.path.exists(source_file_path):
        function_exists_in_source = check_file_for_function(source_file_path)
        if function_exists_in_source:
            print(f"Function '{function_name}' found in source file '{source_file_name}'")
        else:
            print(f"Warning: Function '{function_name}' not found in source file '{source_file_name}'")
    else:
        print(f"Warning: Source file not found: {source_file_path}")


    for folder in all_folders:
        folder_path = os.path.join(base_path, folder)


        for file in os.listdir(folder_path):
            if file.endswith('.sql'):

                if folder == 'Functions' and file == source_file_name:
                    continue

                file_path = os.path.join(folder_path, file)
                if check_file_for_function(file_path):
                    results[folder].append(file)

    return results, function_exists_in_source

def main():

    function_name = "fnGetRolePermissions"

    source_file_name = "fnCheckBulkPermissions.sql"


    results, function_exists_in_source = search_function_in_sql_files(function_name, source_file_name)

    if results is None:
        print("Search could not be completed.")
        return


    function_file = "Not found"
    procedure_file = "Not found"
    table_file = "Not found"


    for folder, files in results.items():
        if files:
            if folder.lower() == 'functions':
                function_file = files[0]
            elif folder.lower() == 'procedure':
                procedure_file = files[0]
            elif folder.lower() == 'tables':
                table_file = files[0]


    output = f'"fnCheckBulkPermissionsor", {function_file}, {procedure_file}, {table_file}'
    print("\nResults in requested format:")
    print(output)


    data = {
        'Function Name': [function_name],
        'Source File': [source_file_name],
    }


    for folder in results.keys():
        data[f'{folder} Files'] = [', '.join(results[folder]) if results[folder] else "Not found"]

    df = pd.DataFrame(data)
    print("\nDetailed results:")
    display(df)


    found_anywhere = any(files for files in results.values())
    if found_anywhere:
        print(f"\nFunction '{function_name}' was found in the following files:")
        for folder, files in results.items():
            if files:
                print(f"- {folder} folder: {', '.join(files)}")
    else:
        print(f"\nFunction '{function_name}' was not found in any files other than the source file.")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Required folder 'Procedure' not found in SearchFile
Function 'fnGetRolePermissions' found in source file 'fnCheckBulkPermissions.sql'

Results in requested format:
"fnCheckBulkPermissionsor", fnGetRolePermissions.sql, Not found, Not found

Detailed results:


,Function Name,Source File,Functions Files,Stored Procedures Files,Tables Files
0,fnGetRolePermissions,fnCheckBulkPermissions.sql,"fnGetRolePermissions.sql, fnGetBulkPermissions...",Not found,Not found



Function 'fnGetRolePermissions' was found in the following files:
- Functions folder: fnGetRolePermissions.sql, fnGetBulkPermissions.sql
